In [1]:
pip install razdel

In [2]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [3]:
from google.colab import files 
uploaded = files.upload()

Saving sents_with_mistakes.txt to sents_with_mistakes.txt


In [4]:
bad = open('sents_with_mistakes.txt', encoding='UTF8')

In [5]:
list_bad = bad.readlines()

In [ ]:
print(list_bad)

In [6]:
from google.colab import files 
uploaded = files.upload()

Saving correct_sents.txt to correct_sents.txt


In [7]:
correct = open('correct_sents.txt', encoding='UTF8')

In [8]:
list_correct = correct.readlines()

In [ ]:
print(list_correct)

In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving wiki_data.txt to wiki_data.txt


In [9]:
import string

In [ ]:
for slot in list_correct:
  print(slot)

Создам словарь правильных слов

In [ ]:
words = []
for slot in list_correct:
  if len(slot) > 1:
    slot = slot.lower()
    list_slots = slot.split()
    for word in list_slots:
      if word.isalpha() and word not in words:
        words.append(word)
print(words)

Создадим словарь модификаций

In [ ]:
words_modif = []
for w in words:
  word_mod = []
  word_mod.append(w)
  if len(w)> 1:
    i_w = 0
    l_w = len(w)
    while l_w != 0:
      string = ''
      for i, b in enumerate(w):
        if i != i_w:
          string += b
      i_w += 1
      l_w -= 1
      word_mod.append(string)
  words_modif.append(word_mod)
print(words_modif)

Создадим словарик потенциальных ошибок

In [ ]:
word_pot_err = []
for slot in list_bad:
  slot = slot.lower()
  list_w = slot.split()
  for word in list_w:
    if word not in words and word not in word_pot_err and word.isalpha():
      word_pot_err.append(word)
print(word_pot_err)

Добавим обработку того, что пользователь забыл букву

In [ ]:
list_correct_forms = []
for pot_err in word_pot_err:
  list_cor_f = []
  list_cor_f.append(pot_err)
  for w in words_modif:
    for w2 in w:
      if w2 == pot_err:
        list_cor_f.append(w[0])
  list_correct_forms.append(list_cor_f)
print(list_correct_forms)

Создадим словарь модификаций из потенциальных ошибок

In [ ]:
words_errors_modif = []
for w in word_pot_err:
  word_mod = []
  word_mod.append(w)
  if len(w)> 1:
    i_w = 0
    l_w = len(w)
    while l_w != 0:
      string = ''
      for i, b in enumerate(w):
        if i != i_w:
          string += b
      i_w += 1
      l_w -= 1
      word_mod.append(string)
  words_errors_modif.append(word_mod)
print(words_errors_modif)

Добавим обработку того, что пользователь добавил лишнюю букву

In [ ]:
for i, w in enumerate(words_errors_modif):
  for w2 in w:
    for slot in words:
      if w2 == slot:
        list_correct_forms[i].append(slot)
print(list_correct_forms)

In [ ]:
for m, slot in enumerate(words_errors_modif):
   for i, w in enumerate(slot):
     if i != 0:
       for slot2 in words_modif:
         for k, w2 in enumerate(slot2):
           if k != 0:
             if w == w2:
               list_correct_forms[m].append(slot2[0])
print(list_correct_forms)

In [ ]:
Добавим алгоритм подсчета расстояние Левенштейна

In [19]:
def distance(a, b):
    n, m = len(a), len(b)
    if n > m:
        a, b = b, a
        n, m = m, n

    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [ ]:
list_correct_forms_sort = []
for slot in list_correct_forms:
  l1 = []
  l2 = []
  l3 = []
  for i, w in enumerate(slot):
    if i != 0:
      l1.append(w)
      l1.append(distance(slot[0], w))
      l2.append(l1)
      l1 = []
  l2 = sorted(l2,  key=lambda l: l[1])
  l3.append(slot[0])
  for s in l2:
    l3.append(s[0])
  list_correct_forms_sort.append(l3)
print(list_correct_forms_sort)

In [80]:
print(list_correct_forms_sort[0][0])

симпатичнейшое


Создадим биграммы

In [21]:
cor = open('correct_sents.txt', encoding='UTF8').read()

In [22]:
corpus = preprocess(cor)

In [23]:
bigrams = Counter()
for cor in corpus:
  bigrams.update(ngrammer(cor, n=2))

In [ ]:
bigrams

Начнем замену

In [24]:
def count (line):
  for key in bigrams.keys():
    st = str(key[0]) + ' ' + str(key[1])
    if line == st:
      return bigrams.get(key)
  else:
     return 0

In [ ]:
for key in bigrams.keys():
    st = str(key[0]) + ' ' + str(key[1])
    print(st)

In [ ]:
  list_new = []
  for line in list_bad:
    n_stroka = ''
    line = line.lower()
    list2 = line.split()
    for i, word in enumerate(list2):
      flag = 1
      for case in list_correct_forms_sort:
        if word.lower() == case[0]:
          flag = 0
          if len(case) == 1:
            n_stroka += case[0] + ' '
            flag = 0
          if len(case) == 2:
            n_stroka += case[1] + ' '
            flag = 0
          if len(case) > 2:
            l2 = []
            for t, s3 in enumerate(case):
              l1 = []
              n = 0
              m = 0
              if t != 0:
                if i - 1 >= 0:
                  st = str(list2[i - 1]) + ' ' + str(s3)
                  n = count(st)
                if i + 1 < len(list2):
                  st2 = str(s3) + ' ' + str(list2[i+1])
                  m = count(st2)
                l1.append(s3)
                l1.append(t)
                l1.append(m+n)
                l2.append(l1)
            l2 = sorted(l2, key = lambda x: x[2])[::-1]
            l4 = []
            for p, sli in enumerate(l2):
              if p == 0:
                maxi = sli[2]
                l4.append(sli)
              else:
                if sli[2] == maxi:
                  l4.append(sli)
            l4 = sorted(l2, key = lambda x: x[1])[::-1]
            n_stroka += str(l4[0][0]) + ' '
            flag = 0
      if flag == 1:
        n_stroka += word + ' '
    list_new.append(n_stroka)
print(list_new)        


In [ ]:
Посчитаем процент правильных слов

In [ ]:
l_c = []
for slot in list_correct:
  list_cor2 = slot.split()
  l_c.append(list_cor2)
print(l_c)

In [ ]:
list_new2 = []
for word in list_new:
  list_new2.append(normalize(word))
print(list_new2)

In [65]:
count_correct = 0
count_total = 0
for i, stroka in enumerate(list_new2):
  for k, word in enumerate(stroka):
    if len(l_c[i]) > k:
      if list_new2[i][k] == l_c[i][k].lower():
        count_correct += 1
      count_total += 1
print(count_correct/count_total)


0.9172331067572971


In [68]:
list_mist = []
for slot in list_bad:
  slot = slot.lower()
  list_w = slot.split()
  list_mist.append(list_w)

Посчитаем процент правильно исправленных ошибок

In [70]:
count_correct_mist = 0
count_total_mist = 0
for i, stroka in enumerate(list_new2):
  for k, word in enumerate(stroka):
    if len(l_c[i]) > k:
      if list_mist[i][k] in word_pot_err:
        if list_new2[i][k] == l_c[i][k].lower():
          count_correct_mist += 1
        count_total_mist += 1
print(count_correct_mist/count_total_mist)

0.6273170731707317


Посчитаем процент исправленных правильных ответов

In [77]:
count_correct_mist_r = 0
count_total_mist_r = 0
for i, stroka in enumerate(list_new2):
  for k, word in enumerate(stroka):
    if len(l_c[i]) > k:
      if list_mist[i][k] in words:
        if list_new2[i][k] != l_c[i][k].lower():
          count_correct_mist_r += 1
        count_total_mist_r += 1
print(count_correct_mist_r/count_total_mist_r)

0.031466064625370396


Задание 2

Создадим триграммы

In [78]:
trigrams = Counter()
for cor in corpus:
  trigrams.update(ngrammer(cor, n=3))

In [79]:
trigrams.most_common(10)

[(('в', 'том', 'что'), 3),
 (('на', 'самом', 'деле'), 3),
 (('в', 'следующий', 'раз'), 3),
 (('да', 'и', 'вообще'), 3),
 (('на', 'мой', 'взгляд'), 3),
 (('об', 'этом', 'и'), 3),
 (('в', 'общем', 'как'), 2),
 (('потому', 'что', 'я'), 2),
 (('не', 'потому', 'что'), 2),
 (('или', 'еще', 'что-то'), 2)]

In [ ]:
for key in trigrams.keys():
    st = str(key[0]) + ' ' + str(key[1]) + ' ' + str(key[2])
    print(st,trigrams.get(key))

In [89]:
def count2 (line):
  for key in trigrams.keys():
    st = str(key[0]) + ' ' + str(key[1]) + ' ' + str(key[2])
    if line == st:
      return trigrams.get(key)
  else:
     return 0

In [ ]:
  list_new = []
  for line in list_bad:
    n_stroka = ''
    line = line.lower()
    list2 = line.split()
    for i, word in enumerate(list2):
      flag = 1
      for case in list_correct_forms_sort:
        if word.lower() == case[0]:
          flag = 0
          if len(case) == 1:
            n_stroka += case[0] + ' '
            flag = 0
          if len(case) == 2:
            n_stroka += case[1] + ' '
            flag = 0
          if len(case) > 2:
            l2 = []
            for t, s3 in enumerate(case):
              l1 = []
              n = 0
              m = 0
              dlina = 0
              if t != 0:
                if i - 2 >= 0:
                  st =  str(list2[i - 1]) + ' ' + str(list2[i - 1]) + ' ' + str(s3)
                  n = count2(st)
                if i + 2 < len(list2):
                  st2 = str(s3) + ' ' + str(list2[i+1]) + ' ' + str(list2[i+2])
                  m = count2(st2)
                if i - 1 >=0 and i + 1 < len(list2):
                  st5 = str(list2[i - 1]) + ' ' + str(s3) + ' ' + str(list2[i + 1])
                  dlina = count2(st5)
                l1.append(s3)
                l1.append(t)
                l1.append(m+n+dlina)
                l2.append(l1)
            l2 = sorted(l2, key = lambda x: x[2])[::-1]
            l4 = []
            for p, sli in enumerate(l2):
              if p == 0:
                maxi = sli[2]
                l4.append(sli)
              else:
                if sli[2] == maxi:
                  l4.append(sli)
            l4 = sorted(l2, key = lambda x: x[1])[::-1]
            n_stroka += str(l4[0][0]) + ' '
            flag = 0
      if flag == 1:
        n_stroka += word + ' '
    list_new.append(n_stroka)
print(list_new) 

In [ ]:
list_new2 = []
for word in list_new:
  list_new2.append(normalize(word))
print(list_new2)

In [92]:
count_correct = 0
count_total = 0
for i, stroka in enumerate(list_new2):
  for k, word in enumerate(stroka):
    if len(l_c[i]) > k:
      if list_new2[i][k] == l_c[i][k].lower():
        count_correct += 1
      count_total += 1
print(count_correct/count_total)

0.9172331067572971


In [93]:
count_correct_mist = 0
count_total_mist = 0
for i, stroka in enumerate(list_new2):
  for k, word in enumerate(stroka):
    if len(l_c[i]) > k:
      if list_mist[i][k] in word_pot_err:
        if list_new2[i][k] == l_c[i][k].lower():
          count_correct_mist += 1
        count_total_mist += 1
print(count_correct_mist/count_total_mist)

0.6273170731707317


In [94]:
count_correct_mist_r = 0
count_total_mist_r = 0
for i, stroka in enumerate(list_new2):
  for k, word in enumerate(stroka):
    if len(l_c[i]) > k:
      if list_mist[i][k] in words:
        if list_new2[i][k] != l_c[i][k].lower():
          count_correct_mist_r += 1
        count_total_mist_r += 1
print(count_correct_mist_r/count_total_mist_r)

0.031466064625370396
